__DD:__ To get ml_metrics installed I had to use Anaconda command prompt and run __pip install ml_metrics__.  I am under the impression that we should avoid using pip in an Anaconda environment but I had no choice.

In [1]:
import pandas as pd
import numpy as np
from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_squared_error, r2_score, mean_absolute_error
from ml_metrics import rmse
import matplotlib.pyplot as plt
from sklearn.datasets import load_boston
from pandas_profiling import ProfileReport

In [2]:
boston = load_boston()

# Convert the matrix to pandas
bos = pd.DataFrame(boston.data)
bos.columns = boston.feature_names
bos['MEDV'] = boston.target
#bos.head()
bos.describe()

,CRIM,ZN,INDUS,CHAS,NOX,RM,AGE,DIS,RAD,TAX,PTRATIO,B,LSTAT,MEDV
count,506.000000,506.000000,506.000000,506.000000,506.000000,506.000000,506.000000,506.000000,506.000000,506.000000,506.000000,506.000000,506.000000,506.000000
mean,3.613524,11.363636,11.136779,0.069170,0.554695,6.284634,68.574901,3.795043,9.549407,408.237154,18.455534,356.674032,12.653063,22.532806
std,8.601545,23.322453,6.860353,0.253994,0.115878,0.702617,28.148861,2.105710,8.707259,168.537116,2.164946,91.294864,7.141062,9.197104
min,0.006320,0.000000,0.460000,0.000000,0.385000,3.561000,2.900000,1.129600,1.000000,187.000000,12.600000,0.320000,1.730000,5.000000
25%,0.082045,0.000000,5.190000,0.000000,0.449000,5.885500,45.025000,2.100175,4.000000,279.000000,17.400000,375.377500,6.950000,17.025000
50%,0.256510,0.000000,9.690000,0.000000,0.538000,6.208500,77.500000,3.207450,5.000000,330.000000,19.050000,391.440000,11.360000,21.200000
75%,3.677083,12.500000,18.100000,0.000000,0.624000,6.623500,94.075000,5.188425,24.000000,666.000000,20.200000,396.225000,16.955000,25.000000
max,88.976200,100.000000,27.740000,1.000000,0.871000,8.780000,100.000000,12.126500,24.000000,711.000000,22.000000,396.900000,37.970000,50.000000


In [3]:
# prof = ProfileReport(bos)
# ProfileReport(bos)

__Step 1:__
Use sklearn.datasets to get the Boston Housing dataset.  Fit a linear regressor to the data as a baseline.  There is no need to do Cross-Validation.  We will simply be exploring the change in results.

In [4]:
train_set = bos.sample(frac=0.7, random_state=100)
test_set = bos[~bos.isin(train_set)].dropna()
# Get the training and testing row indices for later use
train_index = train_set.index.values.astype(int)
test_index = test_set.index.values.astype(int)

# Converting the training and testing datasets back to matrix-formats
X_train = train_set.iloc[:, :-1].values # returns the data; excluding the target
Y_train = train_set.iloc[:, -1].values # returns the target-only
X_test = test_set.iloc[:, :-1].values # ""
Y_test = test_set.iloc[:, -1].values # ""

# Fit a linear regression to the training data
reg = LinearRegression(normalize=True).fit(X_train, Y_train)

#Predict
Y_pred = reg.predict(X_test)

#Get measures
orig_mae = mean_absolute_error(Y_test,Y_pred)
orig_mse = mean_squared_error(Y_test,Y_pred)
orig_rmse_val = rmse(Y_test,Y_pred)
orig_r2 = r2_score(Y_test,Y_pred)

In [5]:
res_frame = pd.DataFrame({'data':'original',
                   'imputation':'none',
                   'mae': orig_mae, 
                   'mse': orig_mse, 
                   'rmse':orig_rmse_val, 
                   'R2':orig_r2,
                   'mae_diff':np.nan,
                   'mse_diff':np.nan,
                   'rmse_diff':np.nan,
                   'R2_diff':np.nan}, index=[0])

__Question 1:__ What is the loss and what are the goodness of fit parameters?  This will be our baseline for comparison.


In [6]:
res_frame

,data,imputation,mae,mse,rmse,R2,mae_diff,mse_diff,rmse_diff,R2_diff
0,original,none,3.604571,24.098505,4.909023,0.70494,NaN,NaN,NaN,NaN


__Step 2:__ (repeat for each percentage value below)
Select 1%, 5% 10%, 20%, 33%, and 50% of your data in a single column [hold that column selection constant throughout all iterations] (Completely at random), replace the original value with a NaN (i.e., “not a number” – ex., np.nan) and then perform an imputation for the missing values.   

__Question 2:__ In each case [1%, 5%, 10%, 20%, 33%, 50%] perform a fit with the imputed data and compare the loss and goodness of fit to your baseline.  [Note: you should have (6) models to compare against your baseline at this point.]


In [7]:
def definedsample(data, fraction, featurename, resultframe):
    in_sample = data.sample(frac=fraction, random_state=99)
    out_sample = data[~data.isin(in_sample)].dropna()
    in_sample[featurename] = np.nan
    in_sample[featurename] = in_sample[featurename].fillna(out_sample[featurename].mean())
    sampleddata = pd.concat([in_sample, out_sample])
    sampleddata = sampleddata.sort_index()
    train_set = sampleddata.iloc[train_index]
    test_set = sampleddata.iloc[test_index]
    X_train = train_set.iloc[:, :-1].values
    Y_train = train_set.iloc[:, -1].values
    X_test = test_set.iloc[:, :-1].values
    Y_test = test_set.iloc[:, -1].values
    
    reg = LinearRegression().fit(X_train, Y_train)
    Y_pred = reg.predict(X_test)
    
    mae = mean_absolute_error(Y_test,Y_pred)
    mse = mean_squared_error(Y_test,Y_pred)
    rmse_val = rmse(Y_test,Y_pred)
    r2 = r2_score(Y_test,Y_pred)
    print(in_sample.iloc)
    temp_frame = pd.DataFrame({'data': str(fraction)+'% missing completely at random: imputed using mean',
                   'imputation':'Mean',
                   'mae': mae, 
                   'mse': mse, 
                   'rmse':rmse_val,
                   'R2':r2,
                   'mae_diff':mae-orig_mae,
                   'mse_diff':mse-orig_mse,
                   'rmse_diff':rmse_val-orig_rmse_val,
                   'R2_diff':r2-orig_r2
                   }, index=[0])
    resultframe = pd.concat([resultframe, temp_frame])
    return resultframe

In [8]:
res_frame = definedsample(bos,.01,'DIS',res_frame)
res_frame = definedsample(bos,.05,'DIS',res_frame)
res_frame = definedsample(bos,.10,'DIS',res_frame)
res_frame = definedsample(bos,.20,'DIS',res_frame)
res_frame = definedsample(bos,.5,'DIS',res_frame)
#Fix the math for the percentages

In [9]:
res_frame

,data,imputation,mae,mse,rmse,R2,mae_diff,mse_diff,rmse_diff,R2_diff
0,original,none,3.604571,24.098505,4.909023,0.704940,NaN,NaN,NaN,NaN
0,0.01% missing completely at random: imputed us...,Mean,3.638791,24.316646,4.931191,0.702269,0.034219,0.218141,0.022168,-0.002671
0,0.05% missing completely at random: imputed us...,Mean,3.553488,23.977540,4.896687,0.706421,-0.051083,-0.120964,-0.012336,0.001481
0,0.1% missing completely at random: imputed usi...,Mean,3.547616,24.027511,4.901786,0.705809,-0.056955,-0.070994,-0.007236,0.000869
0,0.2% missing completely at random: imputed usi...,Mean,3.641010,25.303311,5.030240,0.690188,0.036439,1.204806,0.121217,-0.014752
0,0.5% missing completely at random: imputed usi...,Mean,3.624329,25.662844,5.065851,0.685786,0.019757,1.564339,0.156828,-0.019154


__Step 3:__ Take two columns and create data “Missing at Random” when controlled for a third variable (i.e., if Variable Z is > 30, then Variables X, Y are randomly missing).  Use your preferred imputation method to fill in 10%, 20% and 30% of your missing data.

__DD:__ Function below takes in multiple feature names of our choice. However the Z criterion from instructions I just hard coded for __INDUS < 7__.

In [10]:
def definedsampleXYZ(data, fraction, missingfeaturenames, resultframe):
    in_sample = data.sample(frac=fraction, random_state=99)
    out_sample = data[~data.isin(in_sample)].dropna()
    
    for x in missingfeaturenames:
        in_sample.loc[in_sample['INDUS'] < 7, x] = np.nan
        in_sample[x] = in_sample[x].fillna(out_sample[x].mean())
    
    sampleddata = pd.concat([in_sample, out_sample])
    sampleddata = sampleddata.sort_index()
        
    train_set = sampleddata.iloc[train_index]
    test_set = sampleddata.iloc[test_index]
    X_train = train_set.iloc[:, :-1].values
    Y_train = train_set.iloc[:, -1].values
    X_test = test_set.iloc[:, :-1].values
    Y_test = test_set.iloc[:, -1].values
    
    reg = LinearRegression().fit(X_train, Y_train)
    Y_pred = reg.predict(X_test)
    
    mae = mean_absolute_error(Y_test,Y_pred)
    mse = mean_squared_error(Y_test,Y_pred)
    rmse_val = rmse(Y_test,Y_pred)
    r2 = r2_score(Y_test,Y_pred)
    
    temp_frame = pd.DataFrame({'data': str(fraction)+'% missing at random where INDUS > 7: imputed using mean',
                   'imputation':'Mean',
                   'mae': mae, 
                   'mse': mse, 
                   'rmse':rmse_val,
                   'R2':r2,
                   'mae_diff':mae-orig_mae,
                   'mse_diff':mse-orig_mse,
                   'rmse_diff':rmse_val-orig_rmse_val,
                   'R2_diff':r2-orig_r2
                   }, index=[0])
    resultframe = pd.concat([resultframe, temp_frame])
    #print(in_sample.describe())
    return resultframe

In [11]:
missingXandY = ['DIS', 'NOX']

res_frame = definedsampleXYZ(bos,.1,missingXandY,res_frame)
res_frame = definedsampleXYZ(bos,.2,missingXandY,res_frame)
res_frame = definedsampleXYZ(bos,.3,missingXandY,res_frame)

__Question 3:__ In each case [10%, 20%, 30%] perform a fit with the imputed data and compare the loss and goodness of fit to your baseline.  [Note: you should have (9) models to compare against your baseline at this point.]

In [12]:
res_frame

,data,imputation,mae,mse,rmse,R2,mae_diff,mse_diff,rmse_diff,R2_diff
0,original,none,3.604571,24.098505,4.909023,0.704940,NaN,NaN,NaN,NaN
0,0.01% missing completely at random: imputed us...,Mean,3.638791,24.316646,4.931191,0.702269,0.034219,0.218141,0.022168,-0.002671
0,0.05% missing completely at random: imputed us...,Mean,3.553488,23.977540,4.896687,0.706421,-0.051083,-0.120964,-0.012336,0.001481
0,0.1% missing completely at random: imputed usi...,Mean,3.547616,24.027511,4.901786,0.705809,-0.056955,-0.070994,-0.007236,0.000869
0,0.2% missing completely at random: imputed usi...,Mean,3.641010,25.303311,5.030240,0.690188,0.036439,1.204806,0.121217,-0.014752
0,0.5% missing completely at random: imputed usi...,Mean,3.624329,25.662844,5.065851,0.685786,0.019757,1.564339,0.156828,-0.019154
0,0.1% missing at random where INDUS > 7: impute...,Mean,3.630117,24.335170,4.933069,0.702042,0.025546,0.236665,0.024046,-0.002898
0,0.2% missing at random where INDUS > 7: impute...,Mean,3.611678,24.237506,4.923160,0.703238,0.007107,0.139001,0.014137,-0.001702
0,0.3% missing at random where INDUS > 7: impute...,Mean,3.619581,24.641156,4.963986,0.698296,0.015010,0.542652,0.054963,-0.006644


__Step 4:__  Create a “Missing Not at Random” pattern in which 25% of the data is missing for a single column.
(__concensus:__ remove a quartile)

__Question 4:__ Perform a fit with the imputed data [25%] and compare the loss and goodness of fit to your baseline.  [Note: you should have (10) models to compare against your baseline at this point.]


__DD:__ We use a function to remove the 1st quartile (anything less than 2.100175) from __only the DIS__ feature. This time hard-coding for DIS rather than INDUS.

In [13]:
bos.DIS.describe()

count    506.000000
mean       3.795043
std        2.105710
min        1.129600
25%        2.100175
50%        3.207450
75%        5.188425
max       12.126500
Name: DIS, dtype: float64

In [22]:

#temp.DIS.loc[temp.DIS < 2.100175] = np.nan #<<This works dont fuck it up
#temp['DIS'].loc[temp['DIS'] < 2.100175] = np.nan <<This works too.. dont fuck it up
#temp.DIS.describe()

#temp=bos
# temp.DIS.loc[temp.DIS < 2.100175] = np.nan
# tempmean= temp.DIS.mean()
# temp.DIS = temp.DIS.fillna(tempmean)



In [15]:
#Because we opt to remove the lower quartile we have to handle 
#getting the mean differently. We first drop the quartile and then take the mean which inherently means our model will be terrible depenting on variable importance.

In [28]:
def definedsample25(data, featurename, resultframe):
    data[featurename].loc[data[featurename] < 2.100175] = np.nan
    data[featurename] = data[featurename].fillna(data[featurename].mean())
    sampleddata = data
    sampleddata = sampleddata.sort_index()
    train_set = sampleddata.iloc[train_index]
    test_set = sampleddata.iloc[test_index]
    X_train = train_set.iloc[:, :-1].values
    Y_train = train_set.iloc[:, -1].values
    X_test = test_set.iloc[:, :-1].values
    Y_test = test_set.iloc[:, -1].values
    
    reg = LinearRegression().fit(X_train, Y_train)
    Y_pred = reg.predict(X_test)
    
    mae = mean_absolute_error(Y_test,Y_pred)
    mse = mean_squared_error(Y_test,Y_pred)
    rmse_val = rmse(Y_test,Y_pred)
    r2 = r2_score(Y_test,Y_pred)
   
    temp_frame = pd.DataFrame({'data': '1st quartile missing not at random: imputed using mean',
                   'imputation':'Mean',
                   'mae': mae, 
                   'mse': mse, 
                   'rmse':rmse_val,
                   'R2':r2,
                   'mae_diff':mae-orig_mae,
                   'mse_diff':mse-orig_mse,
                   'rmse_diff':rmse_val-orig_rmse_val,
                   'R2_diff':r2-orig_r2
                   }, index=[0])
    resultframe = pd.concat([resultframe, temp_frame])
    return resultframe    

In [29]:
missingXandY = 'DIS'
res_frame = definedsample25(bos,missingXandY,res_frame)

In [30]:
res_frame

,data,imputation,mae,mse,rmse,R2,mae_diff,mse_diff,rmse_diff,R2_diff
0,original,none,3.604571,24.098505,4.909023,0.704940,NaN,NaN,NaN,NaN
0,0.01% missing completely at random: imputed us...,Mean,3.638791,24.316646,4.931191,0.702269,0.034219,0.218141,0.022168,-0.002671
0,0.05% missing completely at random: imputed us...,Mean,3.553488,23.977540,4.896687,0.706421,-0.051083,-0.120964,-0.012336,0.001481
0,0.1% missing completely at random: imputed usi...,Mean,3.547616,24.027511,4.901786,0.705809,-0.056955,-0.070994,-0.007236,0.000869
0,0.2% missing completely at random: imputed usi...,Mean,3.641010,25.303311,5.030240,0.690188,0.036439,1.204806,0.121217,-0.014752
0,0.5% missing completely at random: imputed usi...,Mean,3.624329,25.662844,5.065851,0.685786,0.019757,1.564339,0.156828,-0.019154
0,0.1% missing at random where INDUS > 7: impute...,Mean,3.630117,24.335170,4.933069,0.702042,0.025546,0.236665,0.024046,-0.002898
0,0.2% missing at random where INDUS > 7: impute...,Mean,3.611678,24.237506,4.923160,0.703238,0.007107,0.139001,0.014137,-0.001702
0,0.3% missing at random where INDUS > 7: impute...,Mean,3.619581,24.641156,4.963986,0.698296,0.015010,0.542652,0.054963,-0.006644
0,1st quartile missing not at random: imputed us...,Mean,3.667832,25.719009,5.071391,0.685099,0.063261,1.620504,0.162368,-0.019841


__Step 5:__ Describe your imputation approach and summarize your findings.  What impact did the missing data have on your baseline model’s performance? 